## Getting Started: a simple discrete-time markov chain (DTMC)

In this notebook we will show the pringles library basics by using a simple discrete-time markov chain (DTMC) as an example. Don't worry if you don't know what a DTMC is, it's just an example.

### The C++ part (user_models/ folder)

In [1]:
!ls -1 user_models/

DTMC.cpp
DTMC.h
Makefile
reg.cpp


The model only has one atomic model called DTMC, defined in *DTMC.h* and *DTMC.cpp* and it is registered for the CD++ simulator in *reg.cpp*.

Then we have the *Makefile* that compiles the user model alongside with the simulator.In the first lines, the path of cd++ sourcecode and the path for the executable are declared. 

In [2]:
!head user_models/Makefile

# Settings
CDPP_PATH 		 = ../../cdpp/src
OUT_DIR          = $(shell pwd)/../bin


KERNEL_SRC       = $(CDPP_PATH)/cd++
OBJ_DIR          = $(OUT_DIR)

MODEL_FLAGS      = -DREGISTER_ATOMICS



### Instatiate simulator

In [ ]:
from pringles.simulator import Simulator

In [ ]:
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='user_models/')

When the simulator is instantiated, two things happen:
    - It checks that there a *cd++* executable in *cdpp_bin_path*. In our case it will be there after executing make in the user_models/ directory
    - It analyses the code in *user_models_dir* searching for the atomic models (by looking at metadata in code comments). They are stored in an AtomicRegistry.

In [ ]:
!tail --line=+9 user_models/DTMC.h | head -n 12

**With the comment starting with @ModelMetadata is parsed and the atomic class is automaticaly created in python**

In [ ]:
print(mySimulator.atomic_registry.discovered_atomics)

In [ ]:
DTMC = mySimulator.atomic_registry.discovered_atomics[0]
print(DTMC)

### Creating the top model in python (equivalent to the static .ma file)

In [ ]:
from pringles.models import Coupled 

When instantiating an atomic, the model parameters are passed as keyword arguments. In this case we have the parameters **r**, **s**, **N** and **initial**.

In [ ]:
a_DTMC = DTMC("a_DTMC", r=0.5, s=0.5, N=20, initial=10)

print("The instance already has an outport, becuase it was read from the metadata.")
print("Number of outports:", len(a_DTMC.outports))
print("Number of inports:", len(a_DTMC.inports))
print("Outport name: ", a_DTMC.outports[0].name)

In [ ]:
top_model = (Coupled(name='top', subcomponents=[a_DTMC])
                .add_outport("out_port")
                .add_coupling(a_DTMC.get_port('currentState_o'), "out_port")
            )
top_model

### Simulate the model

In [ ]:
from pringles.simulator import Simulation
from pringles.utils import VirtualTime

In [ ]:
a_simulation = Simulation(top_model = top_model, duration = VirtualTime.of_hours(1))

# Executes the actual simulation
results = mySimulator.run_simulation(a_simulation)

In [ ]:
display(results.output_df.head(10))

In [ ]:
print(results.logs_dfs.keys(),'\n\n')
display(results.logs_dfs['ParallelRoot'].head())

In [ ]:
import matplotlib.pyplot as plt
from pringles.utils import vtime_decorate

fig, axes = plt.subplots(1, 3, figsize=(20, 10))
results.plot_port('ParallelRoot', 'out_port', axes=vtime_decorate(axes[0]), index=0)
results.plot_port('ParallelRoot', 'out_port', axes=vtime_decorate(axes[1]), index=1)
results.plot_port('ParallelRoot', 'out_port', axes=vtime_decorate(axes[2]), index=2)

### Simulation Persistance

By default, pringles saves all the model (.ma), logs and output files from simulations in temp files.

The user can also indicate the location where these files will be saved when creating the Simulation object:

In [ ]:
a_simulation = Simulation(top_model = top_model,
                          duration = VirtualTime.of_hours(1),
                          working_dir='sim_results/')

# Executes the actual simulation, now in sim_results/ directory
mySimulator.run_simulation(a_simulation)

Besides being returned by `run_simulation`, can also be accessed through the `Simulation` object

In [ ]:
a_simulation.result

All the data from the simulation is saved in a folder inside the working_dir, with a timestamp as a name. The name of the folder can be accessed through `a_simulation.output_dir`

In [ ]:
!ls "$a_simulation.output_dir"

#### What's in the directory of the simulation?

- `logs`, the main log file generated by CD++
- `logsXX`, one log for each component of the simulation, also generated by CD++
- `output`, CD++ output file
- `top_model`, the generated .ma file used as input of the simulation executable
- `simulation.pkl`, a [pickle](https://docs.python.org/3/library/pickle.html) of the simulation object

After running the simulation, a pickle of the simulation is automaticaly generated.

If you wan't to analysis data from old simulation, you can do so by reading the pickle file:

In [ ]:
pickle_path = a_simulation.output_dir + '/simulation.pkl'

recovered_simulation = Simulation.read_pickle(pickle_path)

**Warning:** for this to work you will have to have the Simulator object instanciated so the model Atomics are discovered.

In [ ]:
recovered_simulation.result.output_df.head()